## Query assignments

There are four tables for this assignments: 1. Stations 2. status 3. trips 4. weather

Here are how the tables look like. 

1. Stations table: 

     station_id |               name                |    lat    |    long     | dockcount |   city   | installation 
------------+-----------------------------------+-----------+-------------+-----------+----------+--------------
          
          2 | San Jose Diridon Caltrain Station | 37.329732 | -121.901782 |        27 | San Jose | 2013-08-06
          
          3 | San Jose Civic Center             | 37.330698 | -121.888979 |        15 | San Jose | 2013-08-05
          
          4 | Santa Clara at Almaden            | 37.333988 | -121.894902 |        11 | San Jose | 2013-08-06
          
          5 | Adobe on Almaden                  | 37.331415 |   -121.8932 |        19 | San Jose | 2013-08-05
          
          6 | San Pedro Square                  | 37.336721 | -121.894074 |        15 | San Jose | 2013-08-07


2. Status table: 

 station_id | bikes_available | docks_available |      timestamp      
------------+-----------------+-----------------+---------------------
          2 |              18 |               9 | 2015-09-01 00:00:00
          2 |              18 |               9 | 2015-09-01 00:01:00
          2 |              18 |               9 | 2015-09-01 00:02:00
          2 |              18 |               9 | 2015-09-01 00:03:00
          2 |              18 |               9 | 2015-09-01 00:04:00
    
    

3. Trips table: 

 trip_id | duration |     start_date      |              start_station              | start_terminal |      end_date       |               end_station               | end_terminal | bike_id | subscriber_type | zip_code 
---------+----------+---------------------+-----------------------------------------+----------------+---------------------+-----------------------------------------+--------------+---------+-----------------+----------
 913465  |      746 | 2015-09-01 00:10:00 | San Francisco Caltrain 2 (330 Townsend) |             69 | 2015-09-01 00:23:00 | San Francisco City Hall                 |           58 |     238 | Subscriber      |    94107
 913466  |      969 | 2015-09-01 00:15:00 | Clay at Battery                         |             41 | 2015-09-01 00:31:00 | Washington at Kearny                    |           46 |      16 | Subscriber      |    94133
 913467  |      233 | 2015-09-01 00:15:00 | Davis at Jackson                        |             42 | 2015-09-01 00:19:00 | Commercial at Montgomery                |           45 |     534 | Subscriber      |    94111
 913468  |      213 | 2015-09-01 01:29:00 | Clay at Battery                         |             41 | 2015-09-01 01:32:00 | Steuart at Market                       |           74 |     312 | Subscriber      |    94107
 913469  |      574 | 2015-09-01 01:33:00 | Steuart at Market                       |             74 | 2015-09-01 01:42:00 | San Francisco Caltrain 2 (330 Townsend) |           69 |     279 | Subscriber      |    94107


4. Weather table: 
    
 date    | maxtemperaturef | meantemperaturef | mintemperaturef | maxdewpointf | meandewpointf | mindewpointf | maxhumidity | meanhumidity | minhumidity | maxsealevelpressurein | meansealevelpressurein | minsealevelpressurein | maxvisibilitymiles | meanvisibilitymiles | minvisibilitymiles | maxwindspeedmph | meanwindspeedmph | maxgustspeedmph | precipitationin | cloudcover | events | winddirdegrees |  zip  
------------+-----------------+------------------+-----------------+--------------+---------------+--------------+-------------+--------------+-------------+-----------------------+------------------------+-----------------------+--------------------+---------------------+--------------------+-----------------+------------------+-----------------+-----------------+------------+--------+----------------+-------
 2015-09-01 |              75 |               67 |              58 |           58 |            56 |           54 |          84 |           67 |          49 |                 29.93 |                  29.89 |                 29.86 |                 10 |                  10 |                  6 |              22 |                9 |              26 |               0 |          2 |        |            245 | 94107
 2015-09-02 |              73 |               68 |              62 |           59 |            56 |           54 |          78 |           68 |          57 |                 29.97 |                  29.93 |                  29.9 |                 10 |                  10 |                 10 |              23 |               12 |              26 |               0 |          5 |        |            249 | 94107
    

Questions:
         1. What was the hottest day in our data set? Where was that?
         2. How many trips started at each station?
         3. What's the shortest trip that happened?
         4. What is the average trip duration, by end station?
         5. What if we wanted to also include a count of the number of trips that started in each cities with its latitude,longitude and the name of the city?
         6. What are the three longest trips on rainy days?    
         7. Which station is full most often?
         8. Return a list of stations with a count of number of trips starting at that station but ordered by dock count.
         9. (Challenge) What's the length of the longest trip for each day it rains anywhere?

Answers: 
        1. SELECT zip, max(maxtemperaturef) FROM Weather GROUP BY zip LIMIT 1; 
                 
        2. SELECT start_station, count(*) from Trips group by start_station;

        3. SELECT trip_id, min(duration) AS duration FROM Trips WHERE duration=(SELECT min(duration) FROM Trips) GROUP  BY trip_id ORDER BY min(duration);
        
        4. SELECT avg(duration), end_terminal FROM Trips GROUP BY end_terminal
        
        5. It is easier to do if we break down the question and do it one by one. First we need to find the avergare lat, long for each city.  Then we need to join the this table with the trips table to count the trips started in each city. Here we used Common Table Expressions.

     Solution: WITH table1 AS (SELECT city, avg(lat) lat, avg(long) long from Stations GROUP BY city)
 SELECT t.city, t.lat , t.long, count(*) from table1 t join Stations on t.city = Stations.city 
 JOIN trips on Stations.name = trips.start_station group by 1, 2, 3;
     
     
       6. 
       a) 
WITH rainy as (SELECT DATE(date) rain_date From weather WHERE events = 'Rain' GROUP BY 1 ) SELECT t.trip_id, t.duration, DATE(t.start_date) FROM trips t JOIN rainy r on DATE(t.start_date) = r.rain_date ORDER BY duration DESC LIMIT 3;

       b)
SELECT DATE(r.date) rain_date, t.trip_id, t.duration FROM trips t JOIN weather r on DATE(t.start_date) = DATE(r.date) WHERE r.events = 'Rain' group by 1, 2, 3 ORDER BY duration DESC LIMIT 3;
       
       7. SELECT s1.station_id, s2.name, sum((CASE WHEN s1.docks_available = 0 THEN 1 ELSE 0 END)) busy
      FROM 
      Status s1 join Stations s2 on s1.station_id = s2.station_id
      GROUP BY 1, 2 order by busy DESC;


      8. SELECT t.start_station, count(*), s.dockcount FROM Trips t join Stations s on t.start_station = s.name group by 1, 3 order by s.dockcount DESC;
       
      9. 
       a) 

WITH table1 AS (SELECT max(duration) duration, DATE(start_date) date FROM Trips GROUP BY DATE(start_date))

SELECT t1.duration, t1.date FROM table1 t1 JOIN Weather t2 on t1.date = DATE(t2.date) WHERE t2.events = 'Rain' group by 1, 2 order by t1.duration DESC;

       b)    

WITH rainy AS ( SELECT  DATE(date) weather_date From weather WHERE events = 'Rain'  GROUP BY 1
),rain_trips AS ( SELECT trip_id, duration, DATE(trips.start_date) rain_trips_date FROM trips JOIN rainy on rainy.weather_date = DATE(trips.start_date) ORDER BY duration DESC )

SELECT  rain_trips_date, max(duration) max_duration from rain_trips GROUP BY 1 ORDER BY max_duration DESC;
   